In [0]:
"""
Created on Fri Apr 42 23:49 2020
@author: Héctor & Pati
@subject: DLAS
"""

In [0]:
# Needed imports
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import plotly.express as px

from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

import tensorflow as tf
from tensorflow import keras
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from keras import optimizers
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.vis_utils import plot_model

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


In [0]:
# Load test data
x_test = np.load("/content/gdrive/My Drive/DLAS/mfccs/test/x_test_scaled.npy")
y_test = np.load("/content/gdrive/My Drive/DLAS/mfccs/test/y_test_labeled.npy")

In [0]:
# Load train data
x_train = np.load("/content/gdrive/My Drive/DLAS/x_train_scaled.npy")
y_train = np.load("/content/gdrive/My Drive/DLAS/y_train_labeled.npy")

In [0]:
# LSTM model definition
dense_output_classes = y_train.shape[1]

model = Sequential()
model.add(LSTM(512, input_shape=(x_train.shape[1:]), activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))

#model.add(LSTM(256, activation='sigmoid', return_sequences=True))
#model.add(Dropout(0.2))

model.add(LSTM(512, activation='sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(dense_output_classes, activation='softmax'))

opt = optimizers.RMSprop(lr=0.001)
#opt = optimizers.Adam(learning_rate=0.001)

In [0]:
# Load weights to test them
checkpoint_path = "/content/gdrive/My Drive/DLAS/models/3_LSTM_dropout0.2_hl512_lr0001_RMSProp_b512_200epochs_sigmoid.h5"
model.load_weights(checkpoint_path)

In [0]:
# LSTM model compilation and summary of the model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 512)           1933312   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 4,034,051
Trainable params: 4,034,051
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Evaluate on test data
results = model.evaluate(x_test, y_test)
print('test loss, test acc:', results)


# Evaluate on test data
540/540 [==============================] - 3s 6ms/step
test loss, test acc: [3.7867680591942863, 0.5796296000480652]


In [0]:
# Predict values for test data
y_pred_classes = model.predict_classes(x_test)
y_pred = model.predict(x_test)

# Labels are one hot encoded, transform
rounded_labels = np.argmax(y_test, axis=1)

In [0]:
# Classification report with metrics values
print(classification_report(rounded_labels, y_pred_classes))

              precision    recall  f1-score   support

           0       0.54      0.81      0.65       180
           1       0.59      0.72      0.65       180
           2       0.72      0.21      0.33       180

    accuracy                           0.58       540
   macro avg       0.62      0.58      0.54       540
weighted avg       0.62      0.58      0.54       540



In [0]:
# Printing confusion matrix representation
cm = confusion_matrix(rounded_labels, y_pred_classes)
fig = px.imshow(cm)
fig.show()

In [0]:
# Printing confusion matrix 
cm

array([[145,  20,  15],
       [ 50, 130,   0],
       [ 73,  69,  38]])